# Part I. ETL Pipeline for Pre-Processing the Files

## THE FOLLOWING CODE SHOULD BE RUN FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project
['/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-15-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-22-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-09-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-18-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-04-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-01-events.csv', '/Users/gdegournay/Documents/PycharmProjects/data_engineering_azure/data_modeling/Project/event_data/2018-11-27-events.csv', '/Users/gdegournay/Documents/Pychar

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra tables set up. 

## The data used for setting up the Cassandra's table are in event_datafile_new.csv which contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
# Create the Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_1
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
    print("Keyspace 'udacity_1' created successfully")
except Exception as e:
    print("Error creating keyspace 'udacity_1':", e)


Keyspace 'udacity_1' created successfully


#### Set Keyspace

In [12]:
try:
    session.set_keyspace('udacity_1')
except Exception as e:
    print(e)


### We need to create tables to run three different types of queries. An example of each type of query is given below.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [14]:
# The table structure enables the extraction of the artist's name, the song title, and the song's length. It is tailored to address queries focused on individual listening experiences within # specific item within a session. The table's design facilitates the efficient retrieval of these details, offering a snapshot of the user's listening # choice at that exact moment in their listening history.
query_1 = """
CREATE TABLE IF NOT EXISTS song_info_by_session (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    song_length float,
    PRIMARY KEY (session_id, item_in_session)
)
"""
try:
    session.execute(query_1)
    print("Table 'song_info_by_session' created successfully")
except Exception as e:
    print(e)



                    

Table 'song_info_by_session' created successfully


In [17]:
# We fill the table song_info_by_session with the data from the csv file event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
## TO-DO: Assign the INSERT statements into the `query` variable
    query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, song, song_length) "
    query = query + " VALUES (%s, %s, %s, %s, %s)"    
    for n_row, line in enumerate(csvreader, start=1):
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
print(f"{n_row} rows inserted into table 'song_info_by_session'")

In [19]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, song_length FROM song_info_by_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
    # Printing the results in a readable format
    for row in rows:
        print(f"Artist: {row.artist}, Song: {row.song}, Song Length: {row.song_length}")
except Exception as e:
    print(e)

Artist: Faithless, Song: Music Matters (Mark Knight Dub), Song Length: 495.30731201171875


In [21]:
# The table song_info_user_session is designed to efficiently handle queries like Query 2, which focuses on extracting artist names, song titles (ordered by itemInSession), and user names    # (first and last) for a specific userid and sessionid. We use a partition key composed of the userid and sessionid, and a clustering column itemInSession to sort the results by the order # of the songs played in the session. We use a composite partition key to ensure that the data is distributed evenly across the cluster.
query_2 = """
CREATE TABLE IF NOT EXISTS song_info_by_user_session (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
)
"""

try:
    session.execute(query_2)
    print("Table 'song_info_by_user_session' created successfully")
except Exception as e:
    print(e)
                      

Table 'song_info_by_user_session' created successfully


In [22]:
# Fill table song_info_by_user_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    query = "INSERT INTO song_info_by_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    for n_row, line in enumerate(csvreader, start=1):
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
print(f"{n_row} rows inserted into table 'song_info_by_user_session'")

6820 rows inserted into table 'song_info_by_user_session'


In [23]:
# Select statement to verify the data was entered into the table
query = "SELECT artist, song, first_name, last_name FROM song_info_by_user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
    # Printing the results in a readable format
    for row in rows:
        print(f"Artist: {row.artist}, Song: {row.song}, User: {row.first_name} {row.last_name}")
except Exception as e:
    print(e)

Artist: Down To The Bone, Song: Keep On Keepin' On, User: Sylvie Cruz
Artist: Three Drives, Song: Greece 2000, User: Sylvie Cruz
Artist: Sebastien Tellier, Song: Kilometer, User: Sylvie Cruz
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), User: Sylvie Cruz


In [24]:
# The table user_info_by_song is designed to efficiently handle queries like Query 3, which focuses on extracting user names (first and last) for a specific song.
query_3 = """
CREATE TABLE IF NOT EXISTS user_info_by_song (
    song text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song, user_id)
)
"""

try:
    session.execute(query_3)
    print("Table 'user_info_by_song' created successfully")
except Exception as e:
    print(e)                 

Table 'user_info_by_song' created successfully


In [25]:
# Fill table user_info_by_song
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name) "
    query = query + " VALUES (%s, %s, %s, %s)"
    for n_row, line in enumerate(csvreader, start=1):
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
print(f"{n_row} rows inserted into table 'user_info_by_song'")

6820 rows inserted into table 'user_info_by_song'


In [26]:
# Select statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM user_info_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    # Printing the results in a readable format
    for row in rows:
        print(f"User: {row.first_name} {row.last_name}")
except Exception as e:
    print(e)

User: Jacqueline Lynch
User: Tegan Levine
User: Sara Johnson


### Drop the tables before closing out the sessions

In [27]:
## TO-DO: Drop the table before closing out the sessions
query_drop1 = "DROP TABLE IF EXISTS song_info_by_session"
query_drop2 = "DROP TABLE IF EXISTS song_info_by_user_session"
query_drop3 = "DROP TABLE IF EXISTS user_info_by_song"
try:
    session.execute(query_drop1)
    session.execute(query_drop2)
    session.execute(query_drop3)
    print("Tables dropped successfully")
except Exception as e:
    print(e)


Tables dropped successfully


### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()